# Forecasting Analysis
This forecasting report is provided to analyze staff billing projections.  It provides both individual and group reports.  It also provides a report on past actuals for comparison.  
***

In [1]:
#import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
from business_duration import businessDuration
import holidays as pyholidays
import datetime
from datetime import time, datetime
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nbconvert
#import sys
import dropbox
from api_keys import DROPBOX_APP_KEY, DROPBOX_APP_SECRET, DROPBOX_REFRESH_TOKEN

import ipywidgets as widgets
from ipywidgets import Layout, SelectMultiple
from IPython.display import display, clear_output

pd.options.mode.chained_assignment = None  # suppresses the SettingWithCopyWarning; default='warn'

#Apply global formatting to all floats to only show hundredths
pd.options.display.float_format = '{:,.1f}'.format

In [2]:
# working_time function to calculate working time for each month
holidaylist = pyholidays.UnitedStates()
unit='hour'
starttime=time(9,0,0)
endtime=time(17,0,0)

# define forecasted month
# parameterize in future work
forecast_month = '2023-04'

startDate = pd.to_datetime(forecast_month, format='%Y-%m').strftime('%m-%y')

In [3]:
# Access to dropbox api key and create active instance
dbx_api = dropbox.Dropbox(
    app_key=DROPBOX_APP_KEY,
    app_secret=DROPBOX_APP_SECRET,
    oauth2_refresh_token=DROPBOX_REFRESH_TOKEN,
)

#Read source data - four files
source_folder_path = '/CTE Analytics Projects/test_folder_forecast_analysis/' 

actuals_file_path = source_folder_path + 'Actual_Hours.xlsx'
actuals_download_link = dbx_api.files_get_temporary_link(actuals_file_path).link
actuals_data = pd.read_excel(actuals_download_link)

staff_mapping_file_path = source_folder_path + 'Staff_Mapping.xlsx'
staff_mapping_download_link = dbx_api.files_get_temporary_link(staff_mapping_file_path).link
staff_mapping = pd.read_excel(staff_mapping_download_link)

# Read extracted data that pulled from every spreadsheet using CTE Forecast Extraction.ipynb
extracted_data_file_path = (f'{source_folder_path}ExtractedData/all_forecast_dfs - {forecast_month}.csv')
extracted_data_download_link = dbx_api.files_get_temporary_link(extracted_data_file_path).link
extracted_forecast_data = pd.read_csv(extracted_data_download_link, index_col=0)
all_forecast_dfs = pd.DataFrame(extracted_forecast_data)

# Read extracted data that pulled from every spreadsheet using CTE Forecast Extraction.ipynb
extracted_prop_data_file_path = (f'{source_folder_path}ExtractedData/all_prop_forecast_dfs - {forecast_month}.csv')
extracted_prop_data_download_link = dbx_api.files_get_temporary_link(extracted_prop_data_file_path).link
extracted_prop_forecast_data = pd.read_csv(extracted_prop_data_download_link, index_col=0)
all_prop_forecast_dfs = pd.DataFrame(extracted_prop_forecast_data)

In [4]:
# Create Staff Mapping DF to map names, weekly hours, vacation accrual, and billable baseline
staff_mapping = staff_mapping[['Actuals Names - MANUAL', 'Forecast Names - MANUAL',\
                              'Weekly Hours', '2023 Vacation Accrual', 'Billable Baseline', 'Staffing Group']]
staff_mapping = staff_mapping.rename({'Actuals Names - MANUAL': 'Actuals Names',\
                                    'Forecast Names - MANUAL': 'Forecast Names'}, axis=1)


In [5]:
### Create Forecast DataFrames
#Change Column Headers from datetime to month-date and reorder columns
#And index the month to start the data analysis and drops previous months from the source excel spreadsheets

forecast_data = all_forecast_dfs
mapper = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S').strftime('%m-%y') if (forecast_data.columns.get_loc(x) >= 3) else x
forecast_data.columns = forecast_data.columns.map(mapper)

cols = forecast_data.columns.tolist()
ordered = cols[3:]
ordered.sort(key=lambda date: datetime.datetime.strptime(date, "%m-%y"))
start_column_number = ordered.index(startDate)
ordered_sliced = ordered[start_column_number:]
cols[3:] = ordered_sliced
forecast_data = forecast_data[cols]
forecast_data = forecast_data.fillna(0)

#Correct name errors  (ie - spaces after Soniya's name, Michael Fields to Michael Field)
forecast_data['Staff'] = forecast_data['Staff'].str.replace(r'(^.*Soniya.*$)', 'Soniya Bhagat')
staff_mapping['Forecast Names'] = staff_mapping['Forecast Names'].str.replace(r'(^.*Soniya.*$)', 'Soniya Bhagat')

forecast_data['Staff'] = forecast_data['Staff'].str.replace(r'(^.*Michael Field.*$)', 'Michael Field')

# Create forecast DataFrame
# Drop names and columns
drop_list = ("Associate", "Director of Grants", "Engineering Associate", "Engineering Consultant",\
             "Grant Writer", "Lead Engineering Consultant", "Lead Engineering Associate",\
             "Lead Managing Consultant", "Managing Consultant", "Managing Director", "Managing Director-Engineering",\
             "Senior Engineering Consultant", "Senior Managing Consultant", "Technical Editor")


forecast_time_employee = forecast_data.drop(forecast_data.columns[[0, 1]], axis=1)
forecast_time_employee = forecast_time_employee[~forecast_time_employee['Staff'].isin(drop_list)]

# Group by staff and sum
forecast_df = forecast_time_employee.groupby("Staff").sum()

In [6]:
## CREATE PROPOSAL FORECAST DATAFRAME BY EMPLOYEE
#Change Column Headers from datetime to month-date and reorder columns
#And index the month to start the data analysis and drops previous months from the source excel spreadsheets

prop_forecast_data = all_prop_forecast_dfs
mapper = lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S').strftime('%m-%y') if (prop_forecast_data.columns.get_loc(x) >= 3) else x
prop_forecast_data.columns = prop_forecast_data.columns.map(mapper)

cols2 = prop_forecast_data.columns.tolist()
ordered2 = cols2[3:]
ordered2.sort(key=lambda date: datetime.datetime.strptime(date, "%m-%y"))
start_column_number2 = ordered2.index(startDate)
ordered_sliced2 = ordered2[start_column_number2:]
#ordered_sliced2 = ordered_sliced2[:numberMonths]
cols2[3:] = ordered_sliced2
prop_forecast_data = prop_forecast_data[cols2]
prop_forecast_data = prop_forecast_data.fillna(0)

# Drop names and columns
prop_forecast_time_employee = prop_forecast_data.drop(forecast_data.columns[[0, 1]], axis=1)

# Group by staff and sum
prop_forecast_df = prop_forecast_time_employee.groupby("Staff").sum()

In [7]:
# Employee and staff group setup 
# Initialize employee and number of forecast months to show
employee = 'Jason T Hanlin'

numberMonths = 18

#choose staff members to evaluate; Setup lists of groups in future to select from
def define_staffing_group(staff_mapping, employee):
    staff_name = staff_mapping.loc[staff_mapping['Forecast Names'] == employee, 'Staffing Group'].iloc[0]
    staff = staff_mapping.loc[staff_mapping['Staffing Group'] == staff_name, 'Forecast Names']
    
    return staff_name, staff

staff_name, staff = define_staffing_group(staff_mapping, employee)

#choose column size based off of group size for formatting
col_size = 4

#choose number of months to show for staff evaluations; this could be the same as forecast months above
numMonths = 18


In [8]:
### Dashboard Inputs

def print_dashboard_inputs(employee, staff_name, startDate, numberMonths):
    print(f'Employee: {employee}')
    print(f'Staff group: {staff_name}')
    print(f'Start month for forecast: {startDate}')
    print(f'Number of forecast months: {numberMonths}')
    
#print_dashboard_inputs(employee, staff_name, startDate, numberMonths)

In [9]:
## ACTUALS SECTION
### Actuals DataFrame

# CTE actuals data by selected employee and their projects is available here.  
# Actual data is pulled from timesheet reports and the last month may be incomplete depending on when the report is pulled.  
# This shows what the selected employee has historically been working on.  
# Condition the Actuals data

# Actuals Names can vary from Forecast Names in source data >> map Forecast Names to Actuals Names
name_map = staff_mapping[['Actuals Names', 'Forecast Names']].copy() 
name_map = name_map[name_map['Actuals Names'].notna()]

# Concatonate Names for report from timesheet_report sourced data
actuals_data['Source Name'] = actuals_data['fname'] + ' ' + actuals_data['lname']

# Reset Name to Forecast Name
actuals_data['Source Name'] = actuals_data['Source Name'].map(name_map.set_index('Actuals Names')['Forecast Names'])

# Format DataFrame
actuals_data = actuals_data.rename({'local_date': 'Date', 'hours': 'Hours', 'jobcode_1': 'Service Item'}, axis=1)

In [10]:
# Get Actuals into a Pivot table by Month Date

def get_actuals_df(staff_mapping, actuals_data):
    actual_time_employee = actuals_data[['Date', 'Source Name', 'Hours']]
    actual_time_employee['Date'] = actual_time_employee.loc[:, 'Date'].dt.strftime('%m-%y')

    # Get Actuals into a Pivot table by Month Date
    actuals_pivot = actual_time_employee.pivot_table(
        index='Source Name',
        columns='Date',
        values=['Hours'],
        aggfunc='sum'
        ).stack(level=0)

    actuals_df = actuals_pivot.reset_index()
    actuals_df.drop(columns=['level_1'], inplace = True)
    actuals_df=actuals_df.fillna(0)
    actuals_df=actuals_df.set_index("Source Name")
    
    #re-order columns by month-year
    orderCols = actuals_df.columns.tolist()
    orderCols.sort(key=lambda date: datetime.datetime.strptime(date, "%m-%y"))
    actuals_df = actuals_df[orderCols]
    
    return actuals_df

actuals_df = get_actuals_df(staff_mapping, actuals_data)

In [11]:
#Get actual hours by employee
def get_actuals_byEmployee(actuals_data):
    actual_project_time_employee = actuals_data[['Date', 'Service Item', 'Source Name', 'Hours']]
    actual_project_time_employee['Date'] = actual_project_time_employee.loc[:, 'Date'].dt.strftime('%m-%y')

    actual_project_time_employee = actual_project_time_employee.loc[actual_project_time_employee["Source Name"]==employee]

    actual_hrs_pivot = actual_project_time_employee.pivot_table(
        index='Service Item',
        columns='Date',
        values=['Hours'],
        aggfunc='sum'
        ).stack(level=0)

    actual_projectHrs_df = actual_hrs_pivot.reset_index()
    actual_projectHrs_df.drop(columns=['level_1'], inplace = True)
    actual_projectHrs_df=actual_projectHrs_df.fillna(0)
    actual_projectHrs_df=actual_projectHrs_df.set_index("Service Item")

    #re-order columns by month-year
    orderCols = actual_projectHrs_df.columns.tolist()
    orderCols.sort(key=lambda date: datetime.datetime.strptime(date, "%m-%y"))
    actual_projectHrs_df = actual_projectHrs_df[orderCols]
    return actual_projectHrs_df

actual_projectHrs_df=get_actuals_byEmployee(actuals_data)



In [12]:
# Plot of actual hours by project for employee

def plot_actual_hours_by_project(employee, actual_projectHrs_df):
    plt.figure(figsize=(20, 10))
    plt.title(f"Actual Hours by Project for {employee}", fontsize=22, weight='bold', pad=20)
    
    sns.heatmap(actual_projectHrs_df, cmap='Blues', linewidths=0.30, annot=True, fmt='.0f')
    sns.set(font_scale=1.2)
    
    plt.tick_params(axis='both', which='major', labelsize=14, labelbottom=False, bottom=False, top=False, labeltop=True)
    plt.ylabel(None)
    plt.xlabel(None)
    
    plt.show();
    plt.tight_layout();

#plot_actual_hours_by_project(employee, actual_projectHrs_df);



In [13]:
### Available Billable Time DataFrame
#This section calculates the available time (based on 8 hr days in period, holidays, vacation time) and working time (available time multiplied by baseline billable %) for each employee.  

def working_time(month, year):
    day = 1
    if (int(month) == 12): 
        end_month = 1
        end_year = int(year) + 1
    else:
        end_month = int(month) + 1
        end_year = year

    startdate = pd.to_datetime(f'{year}-{month}-{day}')
    enddate = pd.to_datetime(f'{end_year}-{end_month}-{day}')

    return (businessDuration(startdate,enddate,starttime,endtime,holidaylist=holidaylist,unit=unit))


In [14]:
#create working_time_df and availability_df
working_time_df = pd.DataFrame(columns=forecast_df.columns)
availability_df = pd.DataFrame(columns=forecast_df.columns)

for name in staff_mapping['Forecast Names']:
    namelist = []
    availability = []
    
    vacation = staff_mapping.loc[staff_mapping['Forecast Names']==name,'2023 Vacation Accrual'].iloc[0]
    billable_goal = staff_mapping.loc[staff_mapping['Forecast Names']==name,'Billable Baseline'].iloc[0]
    
    for col in forecast_df.columns:
        month = col[:2]
        year = f'20{col[-2:]}'
    
        time = working_time(month, year)               
            
        namelist.append(billable_goal*(time-vacation/12))
        availability.append(time-vacation/12)
    
    working_time_df.loc[len(working_time_df)] = namelist
    availability_df.loc[len(availability_df)] = availability

working_time_df.index = staff_mapping['Forecast Names']
availability_df.index = staff_mapping['Forecast Names']

#working_time_df
#availability_df


In [15]:
### Forecasting Results Table and Graph for Selected Employee
# This is the summary forecasting results table and chart for the selected employee and number of months.  Use to evaluate how forecasted time compares to billable baseline goal time.  
# Make a table for employee results

def generate_employee_forecast_df(employee, forecast_df, working_time_df, availability_df, numberMonths):
    # if else statement to catch employee that is not in any forecasts and set forecast_billable to zeros
    if employee in forecast_df.index:
        forecast_billable = forecast_df.loc[employee]

    else:
        forecast_billable = pd.Series([0] * len(forecast_df.columns), index=forecast_df.columns)
        forecast_billable.name = employee
        note1 = (f'{employee} is not found in any forecasts!')
        print('\033[1m' + note1)

    # if else statement to catch employee that is not included in staff mapping and set forecast_billable to zeros with exception note
    if employee in working_time_df.index:
        billable_goal = working_time_df.loc[employee]
        available = availability_df.loc[employee]

    else:
        billable_goal = pd.Series([0] * len(working_time_df.columns), index=working_time_df.columns)
        billable_goal.name = employee
        available = pd.Series([0] * len(availability_df.columns), index=availability_df.columns)
        available.name = employee
        note2 = (f'{employee} does not have billable time assigned to staff mapping sheet!')
        print('\033[1m' + note1)
        #sys.exit('\033[1m' + note2)

    #create over_under list
    array1 = np.array(forecast_billable)
    array2 = np.array(billable_goal)
    array3 = np.array(available)

    subtracted_array = np.subtract(array1, array2)
    over_under = list(subtracted_array)

    # create percent availability list
    percent_array = np.divide(np.subtract(array3, array1), array3)*100
    percent_available = list(percent_array)

    #create dataframe
    employee_forecast = pd.DataFrame(columns=forecast_df.columns)
    employee_forecast.loc[len(employee_forecast)] = forecast_billable
    employee_forecast.loc[len(employee_forecast)] = billable_goal
    employee_forecast.loc[len(employee_forecast)] = over_under
    employee_forecast.loc[len(employee_forecast)] = percent_available
    employee_forecast.index = ("Forecast", "Billable Goal", "Over_Under", "% Available")
    employee_forecast_df=employee_forecast.iloc[:,:numberMonths]
    employee_forecast_df.index.name = employee

    #if employee not in working_time_df.index:
    return employee_forecast_df, forecast_billable, billable_goal, available, array2, array3

employee_forecast_df, forecast_billable, billable_goal, available, array2, array3 = generate_employee_forecast_df(employee, forecast_df, working_time_df, availability_df, numberMonths)
  
#employee_forecast_df

In [16]:
# Employee Over_Under Plot
def plot_forecast_analysis(employee, employee_forecast_df):
    plot_data = employee_forecast_df.iloc[2, :]

    # Set x axis and tick locations
    x_axis = np.arange(len(plot_data))
    tick_locations = [value for value in x_axis]

    # Create a list indicating where to write x labels and set figure size to adjust for space
    plt.figure(figsize=(10, 8))
    plt.plot(employee_forecast_df.columns, employee_forecast_df.iloc[0, :], marker="o", color="tab:olive", linewidth=1,
             label='Forecast Hours')
    plt.plot(employee_forecast_df.columns, employee_forecast_df.iloc[1, :], marker="o", color="tab:orange", linewidth=1,
             label='Billable Hours')
    plt.bar(x_axis, plot_data, align="center", color=(plot_data > 0).map({True: 'tab:blue', False: 'tab:red'}))
    plt.xticks(tick_locations, plot_data.index, rotation="vertical", fontsize=14)

    # Set labels for the chart
    plt.title(f"Forecast Hours Over/Under Versus Billable Hours - {employee}", fontsize=18, weight='bold')
    plt.xlabel("Forecast Month", fontsize=14)
    plt.ylabel(f'Over Under (Hours)', fontsize=14)
    values = np.arange(-100, 200, 20)
    plt.yticks(values, fontsize=14)
    plt.axhline(y=0, color='k', linestyle='-')

    plt.legend(loc="upper right")

    # Show Plot
    plt.show()
    plt.tight_layout()
    
#plot_forecast_analysis(employee, employee_forecast_df)

In [17]:
### Forecasted Hours by Project for Staff Member
# This is the forecasted hours by Project for the selected employee and number of months.  
# Use this to see which projects an employee is forecasted to be working on.  
# Can also be used to assess the validity of the monthly forecasting spreadsheets for each project.  

def table_forecast_hours_by_project(employee, forecast_data, forecast_df, numberMonths):
    forecast_project_employee = forecast_data.drop(forecast_data.columns[[0]], axis=1)
    forecast_project_employee = forecast_project_employee.iloc[:,:28]

    # Group project hours for staff member
    project_hrs_df = forecast_project_employee.loc[forecast_project_employee["Staff"]==employee].groupby("Project Name").sum()
    project_hrs_df = project_hrs_df.iloc[:,:numberMonths]

    if employee in forecast_df.index:
        # Plot of hours over/under Baseline Billable Goal
        # heatmap output
        plt.figure(figsize=(20,10))  #Use (20,4) for Jasons staff, about 4 vertical for every 10 projects. Use (20,40 for full staff)
        plt.title(f"Forecasted Hours by Project for {employee}", fontsize=22, weight = 'bold', pad=20)

        #plt.xlabel(f"Green is oversubscribed :: Red is undersubscribed", fontsize=14, style = 'italic')
        sns.heatmap(project_hrs_df, cmap ='Blues', linewidths = 0.30, annot = True, fmt='.0f')
        sns.set(font_scale=1.2)
        plt.tick_params(axis='both', which='major', labelsize=14, labelbottom = False, bottom=False, top = False, labeltop=True)
        plt.ylabel(None, fontsize=20)

        plt.show()
        plt.tight_layout()

    else:
        print('\033[1m' + note1)

#table_forecast_hours_by_project(employee, forecast_data, forecast_df, numberMonths)

In [18]:
## FORECAST FOR STAFFING GROUP
### Forecasting Results Table for a Staffing Group
# This section contains the forecasting results for a the staffing group corresponding to employee and org chart. 

#Use Forecasted Hours Over/Under Baseline chart to see how workload relates to each employees baseline billable goal.  
#Use for making new project assignments to staff (making sure they are busy) but also making sure staff aren't overloaded.


def generate_staff_report(forecast_df, working_time_df, availability_df, staff_mapping, staff, numberMonths):
    staff_df = pd.DataFrame(columns=forecast_df.columns) #this dataframe will show over/under hours for each employee
    staff_availability_df = pd.DataFrame(columns=forecast_df.columns) #this dataframe will show hours available for each employee

    for person in staff:
        # if else statement to catch employee that is not in any forecasts and set forecast_billable to zeros
        if person in forecast_df.index:
            forecast_hours_member = forecast_df.loc[person]
            billable_goal_member = working_time_df.loc[person]
            available_member = availability_df.loc[person]

        else:
            forecast_hours_member = pd.Series([0] * len(forecast_df.columns), index=forecast_df.columns)
            forecast_hours_member.name = person           
            billable_goal_member = staff_mapping.loc[staff_mapping['Forecast Names']==person, 'Billable Baseline'].iloc[0]
            available_member = working_time_df.loc[person] / billable_goal

        # create over_under list
        array6 = np.array(forecast_hours_member)
        array7 = np.array(billable_goal_member)
        array8 = np.array(available_member)

        over_under = list(np.subtract(array6, array7))   # forecasted hours minus billable goal hours
        staff_df.loc[len(staff_df)] = over_under

        # create available hours list and dataframe
        available_hrs = list(np.subtract(array8, array6))  # available hours minus forecasted hours
        staff_availability_df.loc[len(staff_availability_df)] = available_hrs

    staff_df.index = staff
    staff_availability_df.index = staff

    staff_overunder_df = staff_df.iloc[:, :numberMonths]
    staff_avail_hrs_df = staff_availability_df.iloc[:, :numberMonths]

    return staff_overunder_df, staff_avail_hrs_df

staff_overunder_df, staff_avail_hrs_df = generate_staff_report(forecast_df, working_time_df, availability_df, staff_mapping, staff, numberMonths)

#staff_overunder_df
#staff_avail_hrs_df

In [19]:
# Plot of hours over/under Baseline Billable Goal
# heatmap output
def plot_staff_forecast_analysis(staff_overunder_df, staff_name, col_size):
    plt.figure(figsize=(20, col_size))  # Use (20,4) for Jasons staff, about 4 vertical for every 10 people. Use (20,40 for full staff)
    plt.title(f"Forecasted Hours Over/Under Baseline Billable Goal for Group: {staff_name}", fontsize=20, weight='bold', pad=20)

    colors = sns.diverging_palette(20, 250, s=150, l=50, as_cmap=True)
    sns.heatmap(staff_overunder_df, cmap=colors, vmin=-160, vmax=160, linewidths=0.30, annot=True, fmt='.0f')
    sns.set(font_scale=1.2)
    plt.figtext(.65, 0.001, "Blue is oversubscribed :: Red is undersubscribed", fontsize=15, style='italic',
                bbox={"alpha": 0.5, "pad": 5})
    plt.tick_params(axis='both', which='major', labelsize=14, labelbottom=False, bottom=False, top=False,
                    labeltop=True)
    plt.ylabel(None)
    plt.xlabel(None)

    plt.show()
    plt.tight_layout()
    
#plot_staff_forecast_analysis(staff_overunder_df, staff_name, col_size)

In [20]:
# Use Available Hours chart to see if employees could be assigned work regardless of billable time.  
# Can also be used to inform "available hours" assessments required for working with consulting firms on bench projects.  


# Plot of hours available (compared to total available working hours)
# heatmap output
def plot_staff_availability_analysis(staff_avail_hrs_df, staff_name, col_size):
    plt.figure(figsize=(20, col_size))  # Use (20,4) for Jasons staff, about 4 vertical for every 10 people. Use (20,40 for full staff)
    plt.title(f"Available Hours for Group: {staff_name}", fontsize=20, weight='bold', pad=20)

    sns.heatmap(staff_avail_hrs_df, cmap='RdYlGn', linewidths=0.30, annot=True, fmt='.0f')
    sns.set(font_scale=1.2)
    plt.figtext(.65, 0.001, "Green is hours available :: Red is hours undersubscribed", fontsize=15, style='italic',
                bbox={"alpha": 0.5, "pad": 5})
    plt.tick_params(axis='both', which='major', labelsize=14, labelbottom=False, bottom=False, top=False,
                    labeltop=True)
    plt.ylabel(None)
    plt.xlabel(None)

    plt.show()
    plt.tight_layout()
    
#plot_staff_availability_analysis(staff_avail_hrs_df, staff_name, col_size)



In [21]:
#### PENDING PROPOSED PROJECTS FORECAST FOR INDIVIDUAL
## EVALUATE EMPLOYEE INCLUDING PROPOSAL FORECAST

def generate_proposal_forecast(employee, forecast_billable, prop_forecast_df, billable_goal, available):
    # if else statement to catch employee that is not in any budget forecasts and set prop_forecast_billable to zeros
    if employee in prop_forecast_df.index:
        prop_forecast_billable = prop_forecast_df.loc[employee]
    else:
        prop_forecast_billable = pd.Series([0] * len(prop_forecast_df.columns), index=prop_forecast_df.columns)
        prop_forecast_billable.name = employee
        note3 = f'Note that {employee} is not found in any budget proposals'

    forecast_incl_prop_wins = forecast_billable.add(prop_forecast_billable, forecast_billable, fill_value=0)
    forecast_incl_prop_wins = pd.DataFrame(forecast_incl_prop_wins)
    forecast_incl_prop_wins['month'] = forecast_incl_prop_wins.index.str[:2]
    forecast_incl_prop_wins['year'] = forecast_incl_prop_wins.index.str[-2:]
    forecast_incl_prop_wins = forecast_incl_prop_wins.sort_values(by=['year', 'month'])
    forecast_incl_prop_wins = forecast_incl_prop_wins.drop(['year', 'month'], axis=1)

    forecast_incl_prop_wins = forecast_incl_prop_wins[forecast_incl_prop_wins.index.isin(forecast_billable.index)]
    forecast_incl_prop_wins = forecast_incl_prop_wins.squeeze()

    array4 = np.array(forecast_incl_prop_wins)
    subtracted_array_prop = np.subtract(array4, array2)
    over_under_wProp = list(subtracted_array_prop)

    percent_array_prop = np.divide(np.subtract(array3, array4), array3) * 100
    percent_available_wProp = list(percent_array_prop)

    prop_employee_forecast = pd.DataFrame(columns=forecast_df.columns)
    prop_employee_forecast.loc[len(prop_employee_forecast)] = forecast_billable
    prop_employee_forecast.loc[len(prop_employee_forecast)] = forecast_incl_prop_wins
    prop_employee_forecast.loc[len(prop_employee_forecast)] = billable_goal
    prop_employee_forecast.loc[len(prop_employee_forecast)] = over_under_wProp
    prop_employee_forecast.loc[len(prop_employee_forecast)] = percent_available_wProp

    prop_employee_forecast.index = ("Forecast", "Forecast Incl Proposal Wins All", "Billable Goal", "Over Under w Wins",
                                    "% Available w Wins")
    prop_employee_forecast.index.name = employee

    return prop_employee_forecast


prop_employee_forecast = generate_proposal_forecast(employee, forecast_billable, prop_forecast_df, billable_goal, available)
#prop_employee_forecast

In [22]:
# Employee Over_Under Plot
#Plot the bar chart

def plot_prop_employee_forecast(prop_employee_forecast, employee):
    plot_data = prop_employee_forecast.iloc[3, :]  # plot data for over_under bar chart
    # Set x axis and tick locations
    x_axis = np.arange(len(plot_data))
    tick_locations = [value for value in x_axis]

    # Create a list indicating where to write x labels and set figure size to adjust for space
    plt.figure(figsize=(10, 8))
    plt.plot(prop_employee_forecast.columns, prop_employee_forecast.iloc[1, :], marker="o", color="tab:cyan",
             linewidth=1, label='Forecast w Wins All')
    plt.plot(prop_employee_forecast.columns, prop_employee_forecast.iloc[0, :], marker="o", color="tab:olive",
             linewidth=1, label='Forecast Hours')
    plt.plot(prop_employee_forecast.columns, prop_employee_forecast.iloc[2, :], marker="o", color="tab:orange",
             linewidth=1, label='Billable Hours')

    plt.bar(x_axis, plot_data, align="center",
            color=(plot_data > 0).map({True: 'tab:blue', False: 'tab:red'}))
    plt.xticks(tick_locations, plot_data.index, rotation="vertical", fontsize=14)

    # Set labels for the chart
    plt.title(f"Forecast Hours Over/Under Versus Billable Hours - {employee}", fontsize=18, weight='bold')
    plt.xlabel("Forecast Month", fontsize=14)
    plt.ylabel(f'Over Under incl Proposal Wins All (Hours)', fontsize=14)
    values = np.arange(-100, 200, 20)
    plt.yticks(values, fontsize=14)
    plt.axhline(y=0, color='k', linestyle='-')

    plt.legend(loc="upper right")
    
    note3 = f'Note that {employee} is not found in any budget proposals'

    if employee not in prop_forecast_df.index:
        plt.figtext(0.1, -0.1, note3, fontsize=10, style='italic', bbox={"alpha": 0.5, "pad": 5})

    # Show Plot
    plt.show()
    plt.tight_layout()
    
#plot_prop_employee_forecast(prop_employee_forecast, employee)

In [23]:
def plot_proposal_hours(prop_forecast_data, employee, numberMonths):
    prop_forecast_project_employee = prop_forecast_data.drop(prop_forecast_data.columns[[0]], axis=1)
    prop_forecast_project_employee = prop_forecast_project_employee.iloc[:, :numberMonths]

    # Group project hours for staff member
    prop_project_hrs_df = prop_forecast_project_employee.loc[
        prop_forecast_project_employee["Staff"] == employee].groupby("Project Name").sum()
    prop_project_hrs_df = prop_project_hrs_df.iloc[:, :numberMonths]
    prop_project_hrs_df.index.name = f'{employee} Proposal Hrs'

    return prop_project_hrs_df

prop_project_hrs_df = plot_proposal_hours(prop_forecast_data, employee, numberMonths)

#prop_project_hrs_df

In [24]:
#Make a Staff Report
def calculate_proposal_staff_hours(prop_forecast_df, forecast_df, working_time_df, availability_df, staff, numberMonths):
    prop_staff_df = pd.DataFrame(columns=forecast_df.columns)  # This dataframe will show over/under hours for each employee
    prop_staff_availability_df = pd.DataFrame(columns=forecast_df.columns)  # This dataframe will show hours available for each employee

    colTrim = len(working_time_df.columns)
    prop_forecast_df = prop_forecast_df.iloc[:, :colTrim]

    for person in staff:
    # if else statement to catch employee that is not in any forecasts and set forecast_billable to zeros
        if person in prop_forecast_df.index and person in forecast_df.index:
            prop_forecast_hours_member = prop_forecast_df.loc[person]
            pforecast_hours_member = forecast_df.loc[person]

        elif person in forecast_df.index:
            prop_forecast_hours_member = pd.Series([0] * len(prop_forecast_df.columns), index=prop_forecast_df.columns)
            prop_forecast_hours_member.name = person      
            pforecast_hours_member = forecast_df.loc[person]

        else:
            prop_forecast_hours_member = pd.Series([0] * len(prop_forecast_df.columns), index=prop_forecast_df.columns)
            prop_forecast_hours_member.name = person
            pforecast_hours_member = pd.Series([0] * len(prop_forecast_df.columns), index=prop_forecast_df.columns)
            pforecast_hours_member.name = person

        prop_billable_goal_member = working_time_df.loc[person]
        prop_available_member = working_time_df.loc[person]/billable_goal

        #add proposal budget hours to forecast hours
        prop_forecast_hours_member_wins = prop_forecast_hours_member.add(pforecast_hours_member, prop_forecast_hours_member, fill_value=0)

        # Create over_under list
        array6 = np.array(prop_forecast_hours_member_wins)
        array7 = np.array(prop_billable_goal_member)
        array8 = np.array(prop_available_member)

        over_under = list(np.subtract(array6, array7))  # Forecasted hours minus billable goal hours
        prop_staff_df.loc[len(prop_staff_df)] = over_under

        # Create available hours list and dataframe
        available_hrs = list(np.subtract(array8, array6))  # Available hours minus forecasted hours
        prop_staff_availability_df.loc[len(prop_staff_availability_df)] = available_hrs

    prop_staff_df.index = staff
    prop_staff_availability_df.index = staff

    prop_staff_overunder_df = prop_staff_df.iloc[:, :numberMonths]
    prop_staff_avail_hrs_df = prop_staff_availability_df.iloc[:, :numberMonths]

    return prop_staff_overunder_df, prop_staff_avail_hrs_df

prop_staff_overunder_df, prop_staff_avail_hrs_df = calculate_proposal_staff_hours(prop_forecast_df, forecast_df, working_time_df, availability_df, staff, numberMonths)

#prop_staff_overunder_df

In [25]:
# Plot of hours over/under Baseline Billable Goal
# heatmap output
def plot_proposal_staff_overunder(prop_staff_overunder_df, staff_name, col_size):
    plt.figure(figsize=(20, col_size))
    plt.title(f"Forecasted Hours Over/Under with Proposed Budgets for Group: {staff_name}", fontsize=20, weight='bold', pad=20)

    colors = sns.diverging_palette(20, 250, s=150, l=50, as_cmap=True)
    sns.heatmap(prop_staff_overunder_df, cmap=colors, vmin=-160, vmax=160, linewidths=0.30, annot=True, fmt='.0f')
    sns.set(font_scale=1.2)
    plt.figtext(.65, 0.001, "Blue is oversubscribed :: Red is undersubscribed", fontsize=15, style='italic',
                bbox={"alpha": 0.5, "pad": 5})
    plt.tick_params(axis='both', which='major', labelsize=14, labelbottom=False, bottom=False, top=False,
                    labeltop=True)
    plt.ylabel(None)
    plt.xlabel(None)

    plt.show()
    plt.tight_layout()
    
#plot_proposal_staff_overunder(prop_staff_overunder_df, staff_name, col_size)


In [26]:
#to output as html without the code use the following in Terminal CLI:
#jupyter nbconvert --to html --no-input CTE_Forecast_Analysis-With-Extraction.ipynb

#to output as pdf without the code use the following in Terminal:
#jupyter nbconvert --to pdf --no-input CTE_Forecast_Analysis-With-Extraction.ipynb


In [27]:
# Define function to update the dashboard based on the selected employee

employee_input = widgets.Dropdown(
    options = forecast_df.index,
    value = 'Jason T Hanlin',
    description='Employee',
    disabled=False,
    layout = Layout(width='50%', height='80px', display='flex')
);

# Create text widget for output
# employee_output = employee_input

# Define function to bind value of the input to the output variable
#def bind_input_to_output(sender):
#    employee_output.value = employee_input.value

# Tell the text input widget to call bind_input_to_output() on submit
# employee_input.observe(bind_input_to_output, names='value')

# Display input text box widget for input
#employee_input

#employee = employee_output.value[0]

# Create output widget for displaying the plots
output = widgets.Output();

# Define function to update the dashboard based on the selected employee
def update_dashboard(employee):
    with output:
        clear_output(wait=True)
        
        staff_name, staff = define_staffing_group(staff_mapping, employee)
        print_dashboard_inputs(employee, staff_name, startDate, numberMonths)
        
        print('')
        print(f'<<<ACTUALS SECTION>>>')
        print(f'CTE actuals data by selected employee and their projects is available here. Actual data is pulled from timesheet reports and the last month may be incomplete depending on when the report is pulled. This shows what the selected employee has historically been working on.')
        
        #plt.figure(figsize=(8, 6))
        plot_actual_hours_by_project(employee, actual_projectHrs_df)
        #plt.axhline(0, color='gray', linestyle='--')
        #plt.show()
        
        print('')
        print('<<<Forecasting Results Table and Graph for Selected Employee>>>')
        print('This is the summary forecasting results table and chart for the selected employee and number of months.  Use to evaluate how forecasted time compares to billable baseline goal time.')  
        
        #plt.figure(figsize=(8, 6))
        employee_forecast_df, forecast_billable, billable_goal, available, array2, array3 = generate_employee_forecast_df(employee, forecast_df, working_time_df, availability_df, numberMonths)
        
        plt.figure(figsize=(8, 6))
        plot_forecast_analysis(employee, employee_forecast_df)
        #plt.show()
        
        display(employee_forecast_df)  # Display the forecast DataFrame
        
        print('')
        print('<<<Forecasted Hours by Project for Staff Member>>>')
        print('This is the forecasted hours by Project for the selected employee and number of months. Use this to see which projects an employee is forecasted to be working on. Can also be used to assess the validity of the monthly forecasting spreadsheets for each project.')
        
        table_forecast_hours_by_project(employee, forecast_data, forecast_df, numberMonths)
              
        print('')
        print('<<<Forecasting Results Table for a Staffing Group>>>')
        print('This section contains the forecasting results for a the staffing group corresponding to employee and org chart.')
        print('')
        print('Use Forecasted Hours Over/Under Baseline chart to see how workload relates to each employees baseline billable goal.  Use for making new project assignments to staff (making sure they are busy) but also making sure staff aren"t overloaded.')
        staff_overunder_df, staff_avail_hrs_df = generate_staff_report(forecast_df, working_time_df, availability_df, staff_mapping, staff, numberMonths)
        plot_staff_forecast_analysis(staff_overunder_df, staff_name, col_size)
        
        print('Use Available Hours chart to see if employees could be assigned work regardless of billable time. Can also be used to inform "available hours" assessments required for working with consulting firms on bench projects. ')
        plot_staff_availability_analysis(staff_avail_hrs_df, staff_name, col_size)
        
        print('')
        print('<<<PENDING PROPOSED PROJECTS FORECAST FOR INDIVIDUAL>>>')
        print('This section runs the individual forecasts but also includes the "what if" scenario that we win pending projects for proposals that have not been selected, awarded, or contracted yet. For pending proposals to be considered, proposal budgets must be copied into the folder Project Forecasting > Proposal Budget Forecasts. The budgets must be in typical CTE format and must be in the folder prior to running the reports. Please remember to delete the project budget from the folder after it is under contract (and included in the monthly forecasts) or the proposal is not awarded.')
        prop_employee_forecast = generate_proposal_forecast(employee, forecast_billable, prop_forecast_df, billable_goal, available)
        display(prop_employee_forecast)
        plot_prop_employee_forecast(prop_employee_forecast, employee)
        
        prop_project_hrs_df = plot_proposal_hours(prop_forecast_data, employee, numberMonths)
        display(prop_project_hrs_df)
        
        prop_staff_overunder_df, prop_staff_avail_hrs_df = calculate_proposal_staff_hours(prop_forecast_df, forecast_df, working_time_df, availability_df, staff, numberMonths)

        plot_proposal_staff_overunder(prop_staff_overunder_df, staff_name, col_size)
        
        
# Bind the update_dashboard function to the value of the employee_input widget
#widgets.interactive(update_dashboard, employee = employee_output.value[0])

# Bind the update_dashboard function to the value of the employee_input widget
employee_input.observe(lambda change: update_dashboard(change.new), names='value')

# Display the input widget and output widget
display(employee_input)
display(output)

Dropdown(description='Employee', index=21, layout=Layout(display='flex', height='80px', width='50%'), options=…

Output()

<Figure size 640x480 with 0 Axes>